In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Trin 1: Definer Pydantic-modeller for strukturerede outputs

Disse modeller definerer **skemaet**, som agenterne vil returnere. Brug af `response_format` med Pydantic sikrer:
- ✅ Type-sikker dataudtrækning
- ✅ Automatisk validering
- ✅ Ingen parsefejl fra fritekstsvar
- ✅ Nem betinget routing baseret på felter


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Trin 2: Opret værktøjet til hotelbooking

Dette værktøj er det, som **availability_agent** vil kalde for at tjekke, om der er ledige værelser. Vi bruger dekoratoren `@ai_function` til at:
- Konvertere en Python-funktion til et AI-kaldbart værktøj
- Automatisk generere JSON-skema for LLM
- Håndtere parameter-validering
- Muliggøre automatisk aktivering af agenter

Til denne demo:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → Har værelser ✅
- **Alle andre byer** → Ingen værelser ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Trin 3: Definer betingelsesfunktioner til routing

Disse funktioner inspicerer agentens svar og afgør, hvilken vej der skal tages i arbejdsgangen.

**Nøglemønster:**
1. Tjek om beskeden er `AgentExecutorResponse`
2. Pars den strukturerede output (Pydantic-model)
3. Returner `True` eller `False` for at styre routing

Arbejdsgangen vil evaluere disse betingelser på **kanter** for at beslutte, hvilken executor der skal kaldes næste gang.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Trin 4: Opret brugerdefineret Display Executor

Executors er workflow-komponenter, der udfører transformationer eller sideeffekter. Vi bruger `@executor`-dekorationen til at oprette en brugerdefineret executor, der viser det endelige resultat.

**Vigtige begreber:**
- `@executor(id="...")` - Registrerer en funktion som en workflow-executor
- `WorkflowContext[Never, str]` - Typeangivelser for input/output
- `ctx.yield_output(...)` - Returnerer det endelige workflow-resultat


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Trin 5: Indlæs miljøvariabler

Konfigurer LLM-klienten. Dette eksempel fungerer med:
- **GitHub-modeller** (Gratis niveau med GitHub-token)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Trin 6: Opret AI-agenter med strukturerede output

Vi opretter **tre specialiserede agenter**, hver pakket ind i en `AgentExecutor`:

1. **availability_agent** - Tjekker hoteltilgængelighed ved hjælp af værktøjet
2. **alternative_agent** - Foreslår alternative byer (når der ikke er værelser)
3. **booking_agent** - Opmuntrer til booking (når der er værelser tilgængelige)

**Nøglefunktioner:**
- `tools=[hotel_booking]` - Giver værktøjet til agenten
- `response_format=PydanticModel` - Tvinger struktureret JSON-output
- `AgentExecutor(..., id="...")` - Pakker agenten til brug i arbejdsgangen


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Trin 7: Byg arbejdsgangen med betingede forbindelser

Nu bruger vi `WorkflowBuilder` til at konstruere grafen med betinget routing:

**Arbejdsgangsstruktur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Vigtige metoder:**
- `.set_start_executor(...)` - Angiver startpunktet
- `.add_edge(from, to, condition=...)` - Tilføjer en betinget forbindelse
- `.build()` - Afslutter arbejdsgangen


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Trin 8: Kør Test Case 1 - By UDEN Tilgængelighed (Paris)

Lad os teste **ingen tilgængelighed**-scenariet ved at forespørge hoteller i Paris (som ikke har nogen værelser i vores simulation).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Trin 9: Kør testtilfælde 2 - By MED tilgængelighed (Stockholm)

Lad os nu teste **tilgængeligheds**-stien ved at anmode om hoteller i Stockholm (som har værelser i vores simulation).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Vigtige Punkter og Næste Skridt

### ✅ Hvad Du Har Lært:

1. **WorkflowBuilder Mønster**
   - Brug `.set_start_executor()` til at definere startpunktet
   - Brug `.add_edge(from, to, condition=...)` til betinget routing
   - Kald `.build()` for at færdiggøre workflowet

2. **Betinget Routing**
   - Betingelsesfunktioner inspicerer `AgentExecutorResponse`
   - Analyser strukturerede outputs for at træffe routingbeslutninger
   - Returner `True` for at aktivere en edge, `False` for at springe den over

3. **Værktøjsintegration**
   - Brug `@ai_function` til at konvertere Python-funktioner til AI-værktøjer
   - Agenter kalder værktøjer automatisk, når det er nødvendigt
   - Værktøjer returnerer JSON, som agenter kan analysere

4. **Strukturerede Outputs**
   - Brug Pydantic-modeller til type-sikker dataudtrækning
   - Sæt `response_format=MyModel` ved oprettelse af agenter
   - Analyser svar med `Model.model_validate_json()`

5. **Brugerdefinerede Executors**
   - Brug `@executor(id="...")` til at oprette workflow-komponenter
   - Executors kan transformere data eller udføre sideeffekter
   - Brug `ctx.yield_output()` til at producere workflow-resultater

### 🚀 Anvendelser i Virkeligheden:

- **Rejsebooking**: Tjek tilgængelighed, foreslå alternativer, sammenlign muligheder
- **Kundeservice**: Rout baseret på problemtype, sentiment, prioritet
- **E-handel**: Tjek lagerstatus, foreslå alternativer, behandl ordrer
- **Indholdsmoderering**: Rout baseret på toksicitetsniveauer, brugerflag
- **Godkendelsesworkflows**: Rout baseret på beløb, brugerrolle, risikoniveau
- **Flertrinsbehandling**: Rout baseret på datakvalitet, fuldstændighed

### 📚 Næste Skridt:

- Tilføj mere komplekse betingelser (flere kriterier)
- Implementer loops med workflow-tilstandsadministration
- Tilføj under-workflows for genanvendelige komponenter
- Integrer med rigtige API'er (hotelbooking, lagersystemer)
- Tilføj fejlhåndtering og fallback-stier
- Visualiser workflows med de indbyggede visualiseringsværktøjer



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal det bemærkes, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
